In [9]:
import pandas as pd

In [18]:
csv = 'DATA_GEMA.csv'

data= pd.read_csv(csv)
data.head()

,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity,designation,id
0,Teso La Monja,Tinto,2013,4.9,58,Espana,Toro,995.00,Toro Red,5.0,3.0,Tinto 2013,1
1,Artadi,Vina El Pison,2018,4.9,31,Espana,Vino de Espana,313.50,Tempranillo,4.0,2.0,Vina El Pison 2018,2
2,Vega Sicilia,Unico,2009,4.8,1793,Espana,Ribera del Duero,324.95,Ribera Del Duero Red,5.0,3.0,Unico 2009,3
3,Vega Sicilia,Unico,1999,4.8,1705,Espana,Ribera del Duero,692.96,Ribera Del Duero Red,5.0,3.0,Unico 1999,4
4,Vega Sicilia,Unico,1996,4.8,1309,Espana,Ribera del Duero,778.06,Ribera Del Duero Red,5.0,3.0,Unico 1996,5


In [27]:
# 1. Crear la tabla `wines`
wines = data[['id', 'wine', 'price', 'rating', 'body', 'acidity', 'num_reviews']].copy()
wines.drop_duplicates(subset=['id'], inplace=True)


In [28]:
# Validar valores numéricos
wines['price'] = wines['price'].apply(lambda x: max(x, 0) if pd.notnull(x) else None)
wines['rating'] = wines['rating'].apply(lambda x: max(min(x, 100), 0) if pd.notnull(x) else None)

In [29]:
wines.head()

,id,wine,price,rating,body,acidity,num_reviews
0,1,Tinto,995.00,4.9,5.0,3.0,58
1,2,Vina El Pison,313.50,4.9,4.0,2.0,31
2,3,Unico,324.95,4.8,5.0,3.0,1793
3,4,Unico,692.96,4.8,5.0,3.0,1705
4,5,Unico,778.06,4.8,5.0,3.0,1309


In [30]:
# 2. Crear la tabla `winery`
winery = data[['winery', 'id', 'region']].drop_duplicates(subset=['winery']).reset_index(drop=True)
winery['winery_id'] = winery.index + 1  # Generar ID único para cada bodega
winery.rename(columns={'id': 'wine_id'}, inplace=True)
winery.head()


,winery,wine_id,region,winery_id
0,Teso La Monja,1,Toro,1
1,Artadi,2,Vino de Espana,2
2,Vega Sicilia,3,Ribera del Duero,3
3,Pago de Carraovejas,12,Ribera del Duero,4
4,Toro Albala,15,Montilla-Moriles,5


In [32]:
region = data[['region', 'country']].drop_duplicates().reset_index(drop=True)
region['region_id'] = region.index + 1
winery = pd.merge(winery, region, on='region', how='left')
region.rename(columns={'region': 'nombreRegion'}, inplace=True)
region.head()


,nombreRegion,country,region_id
0,Toro,Espana,1
1,Vino de Espana,Espana,2
2,Ribera del Duero,Espana,3
3,Montilla-Moriles,Espana,4
4,Jumilla,Espana,5


In [33]:
wine_type = data[['id', 'type']].drop_duplicates().reset_index(drop=True)
wine_type['type_id'] = wine_type.index + 1  # Generar ID único para cada tipo
wine_type.rename(columns={'id': 'wine_id'}, inplace=True)  # Relacionar con `wines`
wine_type.head()

,wine_id,type,type_id
0,1,Toro Red,1
1,2,Tempranillo,2
2,3,Ribera Del Duero Red,3
3,4,Ribera Del Duero Red,4
4,5,Ribera Del Duero Red,5


In [37]:
wines.to_csv('wines_sql.csv', index=False)
winery.to_csv('winery_sql.csv', index=False)
region.to_csv('region_sql.csv', index=False)
wine_type.to_csv('wine_type_sql.csv', index=False)